In [1]:
import pandas as pd
import boto3
from io import BytesIO
from datetime import datetime
from openpyxl import load_workbook
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from  src.calc_primas import calcular_edad
from src.calc_primas import obtener_lista_nombre_bases
from src.calc_primas import obtener_base_parametros
from src.calc_primas import obtener_base_cuotas
from src.calc_primas import obtener_base_emisiones
from src.calc_primas import obtener_base_historico

In [2]:
bucket_name = "itam-analytics-grb"
s3 = boto3.client("s3")

In [3]:
# Rutas
ruta_base_asegurados = "coco/data/solicitudes/Base_Datos/"
ruta_parametros = "coco/data/solicitudes/parametros/parametros.xlsx"
ruta_cuotas = "coco/modelo/experiencia_global.xlsx"
ruta_emisiones = "coco/data/emisiones.xlsx"
ruta_historico_cotizaciones = "coco/data/cotizaciones.xlsx"


In [4]:
obtener_lista_nombre_bases(ruta_base_asegurados, bucket_name)

['coco/data/solicitudes/Base_Datos/EmpresaA.xlsx',
 'coco/data/solicitudes/Base_Datos/EmpresaB.xlsx',
 'coco/data/solicitudes/Base_Datos/EmpresaC.xlsx',
 'coco/data/solicitudes/Base_Datos/EmpresaD.xlsx',
 'coco/data/solicitudes/Base_Datos/EmpresaE.xlsx']

In [9]:
len(obtener_base_parametros(ruta_parametros, bucket_name))

5

In [6]:
obtener_base_cuotas(ruta_cuotas, bucket_name)

,Edad,Fallecimiento,MA,BPAI
0,15,1.21,0.52,0.19
1,16,1.21,0.52,0.19
2,17,1.23,0.52,0.19
3,18,1.25,0.52,0.22
4,19,1.26,0.52,0.25
...,...,...,...,...
56,71,27.70,0.00,0.00
57,72,31.60,0.00,0.00
58,73,36.08,0.00,0.00
59,74,41.13,0.00,0.00


In [7]:
obtener_base_emisiones(ruta_emisiones, bucket_name)

,Poliza,InicioVigencia,FinVigencia,PrimaNeta,Siniestralidad
0,3300,2025-04-21,2025-12-31,326036.90,0.2
1,3299,2025-05-01,2026-05-01,20900000.00,0.3
2,3298,2025-04-23,2026-04-23,2952382.48,0.0
3,3297,2025-05-01,2026-05-01,1941687.43,0.2
4,3296,2025-04-18,2026-04-18,389871.20,1.0
...,...,...,...,...,...
81,857,2025-04-21,2026-04-21,208153.50,0.9
82,537,2025-02-07,2026-02-07,98936.50,0.3
83,341,2025-02-15,2026-02-15,17483.00,0.6
84,260,2025-07-01,2026-07-01,50000.00,0.3


In [10]:
len(obtener_base_historico(ruta_historico_cotizaciones, bucket_name))

557

In [ ]:
MESES = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto", 
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

RECARGOS_FORMA_PAGO = {
    "anual": {"rpf": 0.0, "num_recibos": 1},
    "semestral": {"rpf": 0.037, "num_recibos": 2},
    "trimestral": {"rpf": 0.055, "num_recibos": 4},
    "mensual": {"rpf": 0.065, "num_recibos": 12}
}

DESCUENTOS_COMISION = {
    0.20: 0.00, 0.19: 0.02, 0.18: 0.03, 0.17: 0.04, 0.16: 0.06,
    0.15: 0.07, 0.14: 0.09, 0.13: 0.10, 0.12: 0.12, 0.11: 0.13,
    0.10: 0.15, 0.09: 0.16, 0.08: 0.18, 0.07: 0.19, 0.06: 0.21, 0.05: 0.22
}

def obtener_nombre_mes(fecha):
    """
    Función que convierte fecha a nombre del mes en español
    
    Parameters:
        fecha: Fecha en formato datetime o string
        
    Returns:
        str: Nombre del mes en español
    """
    mes_numero = pd.to_datetime(fecha).month
    return MESES[mes_numero]


def obtener_parametros_forma_pago(forma_pago):
    """
    Obtiene RPF y número de recibos según forma de pago
    
    Parameters:
        forma_pago (str): Forma de pago (Anual, Semestral, Trimestral, Mensual)
        
    Returns:
        dict: Diccionario con 'rpf' y 'num_recibos'
    """
    return RECARGOS_FORMA_PAGO.get(forma_pago.lower(), RECARGOS_FORMA_PAGO["anual"])


def obtener_descuento_comision(comision):
    """
    Obtiene descuento según nivel de comisión
    
    Parameters:
        comision (float): Nivel de comisión (0.05 a 0.20)
        
    Returns:
        float: Descuento correspondiente
    """
    return DESCUENTOS_COMISION.get(comision, 0.0)


def obtener_nombre_cobertura(codigo_cobertura):
    """
    Convierte código de cobertura a nombre completo
    
    Parameters:
        codigo_cobertura (str): Código de cobertura (F, FMA, FBPAI, FMABPAI)
        
    Returns:
        str: Nombre completo de la cobertura
    """
    coberturas_map = {
        "F": "FALLECIMIENTO",
        "FMA": "FALLECIMIENTO Y MUERTE ACCIDENTAL",
        "FBPAI": "FALLECIMIENTO E INVALIDEZ TOTAL",
        "FMABPAI": "FALLECIMIENTO, MUERTE ACCIDENTAL E INVALIDEZ TOTAL"
    }
    return coberturas_map.get(codigo_cobertura, codigo_cobertura)


In [ ]:
def prima_experiencia_global(ruta_carpeta_base_datos:str, 
                             ruta_archivo_parametros:str, 
                             ruta_cuotas:str, 
                             ruta_carpeta_calculos:str, 
                             ruta_emisiones:str, 
                             ruta_historico_cotizaciones:str, 
                             nombre_bucket:str):
    s3 = boto3.client("s3")

    # Cargamos bases de datos
    lista_archivos_base_datos = obtener_lista_nombre_bases(ruta_carpeta_base_datos, nombre_bucket) # Lista de archivos de bases de datos
    df_parametros = obtener_base_parametros(ruta_archivo_parametros, nombre_bucket) # df con párametros de cotización
    df_cuotas = obtener_base_cuotas(ruta_cuotas, nombre_bucket) # df con cuotas al millar
    df_emisiones = obtener_base_emisiones(ruta_emisiones, nombre_bucket) # df con emisiones
    df_hist_cotizaciones = obtener_base_historico(ruta_historico_cotizaciones, nombre_bucket) # df con histórico de cotizaciones

    cotizacion = {
        "Contratante": [],
        "Coberturas": [],
        "SumaAsegurada": [],
        "Administracion": [],
        "Agente": [],
        "Comision": [],
        "FormaPago": [],
        "Inicio": [],
        "Fin": [],
        "Renovacion": [],
        "Poliza": [],
        "Prima": [],
        "EdadPromedio": [],
        "SAMI": [],
        "Asegurados": [],
        "Ticket": [],
        "Mes": [],
        "Oficina": [],
        "Evento": []
    }

    MESES ={
        1: "Enero",
        2: "Febrero",
        3: "Marzo",
        4: "Abril",
        5: "Mayo",
        6: "Junio",
        7: "Julio",
        8: "Agosto",
        9: "Septiembre",
        10: "Octubre",
        11: "Noviembre",
        12: "Diciembre"
    }
    
    for j in range(0,len(df_parametros)):
        
        ruta_base_datos_asegurados = ""
        ruta_guardar_calculo = ""
        rpf = 0
        desc = 0
        prima = 0
        ticket = len(df_hist_cotizaciones) + j + 1
        
        cotizacion["Contratante"].append(df_parametros["Contratante"][j])
        cotizacion["Coberturas"].append(df_parametros["Coberturas"][j])
        cotizacion["SumaAsegurada"].append(df_parametros["SumaAsegurada"][j])
        cotizacion["Administracion"].append(df_parametros["Administracion"][j])
        cotizacion["Agente"].append(df_parametros["Agente"][j])
        cotizacion["Comision"].append(df_parametros["Comision"][j])
        cotizacion["FormaPago"].append(df_parametros["FormaPago"][j])
        cotizacion["Inicio"].append(df_parametros["Inicio"][j])
        cotizacion["Fin"].append(df_parametros["Fin"][j])
        cotizacion["Renovacion"].append(df_parametros["Renovacion"][j])
        cotizacion["Poliza"].append(df_parametros["Poliza"][j])
        cotizacion["Ticket"].append(ticket)
        cotizacion["Oficina"].append(df_parametros["Oficina"][j])

        mes_numero = pd.to_datetime(df_parametros["Inicio"][j]).month
        mes = MESES[mes_numero]  # Usar el diccionario para obtener el nombre del mes
        cotizacion["Mes"].append(mes)

        ruta_base_datos_asegurados = lista_archivos_base_datos[j]

        #Carga de la base de datos de asegurados
        response = s3.get_object(Bucket=nombre_bucket, Key=ruta_base_datos_asegurados)
        content = response['Body'].read()
        df_calculo = pd.read_excel(BytesIO(content), engine='openpyxl')
        
        fecha_corte = pd.to_datetime(df_parametros["Inicio"][j])
        df_calculo["Edad"] = df_calculo["Fecha de Nacimiento"].apply(lambda x: calcular_edad(x, fecha_corte))

        if df_parametros["FormaPago"][j] == "Anual".lower():
            rpf = 0
            num_recibos = 1
        elif df_parametros["FormaPago"][j] == "Semestral".lower():
            rpf = 0.037
            num_recibos = 2
        elif df_parametros["FormaPago"][j] == "Trimestral".lower():
            rpf = 0.055
            num_recibos = 4
        elif df_parametros["FormaPago"][j] == "Mensual".lower():
            rpf = 0.065
            num_recibos = 12

        if df_parametros["Comision"][j] == .2:
            desc = 0
        elif df_parametros["Comision"][j] == .19:
            desc = 0.02
        elif df_parametros["Comision"][j] == .18:
            desc = 0.03
        elif df_parametros["Comision"][j] == .17:
            desc = 0.04
        elif df_parametros["Comision"][j] == .16:
            desc = 0.06
        elif df_parametros["Comision"][j] == .15:
            desc = 0.07
        elif df_parametros["Comision"][j] == .14:
            desc = 0.09
        elif df_parametros["Comision"][j] == .13:
            desc = 0.10
        elif df_parametros["Comision"][j] == .12:
            desc = 0.12
        elif df_parametros["Comision"][j] == .11:
            desc = 0.13
        elif df_parametros["Comision"][j] == .10:
            desc = 0.15
        elif df_parametros["Comision"][j] == .09:
            desc = 0.16
        elif df_parametros["Comision"][j] == .08:
            desc = 0.18
        elif df_parametros["Comision"][j] == .07:
            desc = 0.19
        elif df_parametros["Comision"][j] == .06:
            desc = 0.21
        elif df_parametros["Comision"][j] == .05:
            desc = 0.22

        if df_parametros["Coberturas"][j] == "F":
            df_calculo = df_calculo.merge(df_cuotas[["Edad","Fallecimiento"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()
        elif df_parametros["Coberturas"][j] == "FMA":
            df_calculo = df_calculo.merge(df_cuotas[["Edad","Fallecimiento","MA"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            df_calculo["MA"] = df_calculo["MA"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["MA"].sum()
        elif df_parametros["Coberturas"][j] == "FBPAI":
            df_calculo = df_calculo.merge(df_cuotas[["Edad","Fallecimiento","BPAI"]], on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            df_calculo["BPAI"] = df_calculo["BPAI"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["BPAI"].sum()
        elif df_parametros["Coberturas"][j] == "FMABPAI":
            df_calculo = df_calculo.merge(df_cuotas, on="Edad", how="left")
            df_calculo["Fallecimiento"] = df_calculo["Fallecimiento"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            df_calculo["MA"] = df_calculo["MA"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            df_calculo["BPAI"] = df_calculo["BPAI"]*(1-desc)*(1+rpf)*df_parametros["SumaAsegurada"][j]/1000
            prima = df_calculo["Fallecimiento"].sum()+df_calculo["BPAI"].sum()+df_calculo["MA"].sum()

        if df_parametros["Renovacion"][j] == "Si":
            siniestralidad = df_emisiones.loc[df_emisiones["Poliza"] == df_parametros["Poliza"][j], "Siniestralidad"].values[0]
            
            if siniestralidad < 0.50:
                cotizacion["Prima"].append(prima)
                cotizacion["Evento"].append("na")
            else:
                cotizacion["Prima"].append("La siniestralidad está desviada, consulte a un suscriptor")
                cotizacion["Evento"].append("Fuera de política")
                prima = "La siniestralidad está desviada, consulte a un suscriptor"
            
        else:
            cotizacion["Prima"].append(prima)
            cotizacion["Evento"].append("na")            
            
            
        cotizacion["EdadPromedio"].append(df_calculo["Edad"].mean())
        cotizacion["SAMI"].append(df_parametros["SumaAsegurada"][j])
        cotizacion["Asegurados"].append(df_calculo["Edad"].count())

        #Guardar base de datos con el calculo de la prima
        file_key = ruta_carpeta_calculos + df_parametros["Contratante"][j] + ".xlsx"
                # Convertir el DataFrame a un archivo Excel en memoria
        buffer = BytesIO()
        df_calculo.to_excel(buffer, index=False, engine='openpyxl')
        buffer.seek(0)  # volver al inicio del buffer
                # Subir el archivo al bucket
        s3.upload_fileobj(buffer, nombre_bucket, file_key)

        #Creación pdf
        bucket_name = nombre_bucket
        imagen_s3_key = "coco/logo_SegurosDelValle.png"
        pdf_s3_key = "coco/solicitudes/formatos_pdf/" + df_parametros["Contratante"][j] + ".pdf"
        imagen_buffer = BytesIO()
        s3.download_fileobj(bucket_name, imagen_s3_key, imagen_buffer)   #Carga del logo
        imagen_buffer.seek(0)
        pdf_buffer = BytesIO()   # Crear PDF en memoria
        c = canvas.Canvas(pdf_buffer)
        # Campos variables del pdf
        contratante = df_parametros["Contratante"][j]
        if df_parametros["Coberturas"][j] == "F":
            coberturas = "FALLECIMIENTO"
        elif df_parametros["Coberturas"][j] == "FMA":
            coberturas = "FALLECIMIENTO Y MUERTE ACCIDENTAL"
        elif df_parametros["Coberturas"][j] == "FBPAI":
            coberturas = "FALLECIMIENTO E INVALIDEZ TOTAL"
        elif df_parametros["Coberturas"][j] == "FMABPAI":
            coberturas = "FALLECIMIENTO, MUERTE ACCIDENTAL E INVALIDEZ TOTAL"
        suma_asegurada = df_parametros["SumaAsegurada"][j]
        edad_promedio = df_calculo["Edad"].mean()
        administracion = df_parametros["Administracion"][j]
        SAMI = df_parametros["SumaAsegurada"][j]
        agente = df_parametros["Agente"][j]
        vigencia = str(df_parametros["Inicio"][j]) + "-" + str(df_parametros["Fin"][j])
        #prima = prima
        forma_pago = df_parametros["FormaPago"][j]
        asegurados = df_calculo["Edad"].count()
        # Insertar imagen en la parte superior izquierda (ajustar tamaño/posición si es necesario)
        imagen = ImageReader(imagen_buffer)
        c.drawImage(imagen, x=30, y=740, width=200, height=75)
        # Agregar texto
        c.drawString(50, 700, "DESGLOSE DE ESTUDIO DE SEGURO DE VIDA GRUPO")
        c.drawString(50, 685, f"CONTRATANTE: {contratante}")
        c.drawString(50, 670, f"COBERTURAS: {coberturas}")
        c.drawString(50, 655, f"SUMA ASEGURADA: $ {suma_asegurada:,.2f}")
        c.drawString(50, 640, f"EDAD PROMEDIO: {edad_promedio}")
        c.drawString(50, 625, f"SISTEMA DE ADMINISTRACIÓN: {administracion}")
        c.drawString(50, 610, f"SAMI: $ {SAMI:,.2f}")
        c.drawString(50, 595, "CONTRIBUTORIO: NO CONTRIBUTORIO")
        c.drawString(50, 580, "DIVIDENDOS: SIN DIVIDENDOS")
        c.drawString(50, 565, f"AGENTE: {agente}")
        c.drawString(50, 550, f"VIGENCIA: {vigencia}")
        if isinstance(prima, str):
            c.drawString(50, 535, f"PRIMA:  {prima}")
        else:
            c.drawString(50, 535, f"PRIMA: $ {prima:,.2f}")
        c.drawString(50, 520, f"FORMA DE PAGO: {forma_pago}")
        if isinstance(prima, str):
            c.drawString(50, 505, f"PRIMA:  {prima}")
        else:
            c.drawString(50, 505, f"PRIMER RECIBO Y SUBSECUENTES: $ {prima/num_recibos:,.2f}")
        c.drawString(50, 490, f"ASEGURADOS: {asegurados}")
        c.drawString(50, 475, "EDAD DE ACEPTACIÓN PARA LA COBERTURA DE FALLECIMIENTO: HASTA 75 AÑOS")
        c.drawString(50, 460, "EDAD DE ACEPTACIÓN PARA LA COBERTURA DE MUERTE ACCIDENTAL: HASTA 69 AÑOS")
        c.drawString(50, 445, "EDAD DE ACEPTACIÓN PARA LA COBERTURA DE INVALIDEZ TOTAL: HASTA 64 AÑOS")
        c.drawString(50, 415, "ESTA COTIZACIÓN NO REPRESENTA COMPROMISO DE COBERTURA ALGUNA Y TIENE")
        c.drawString(50, 403, "VIGENCIA DE 30 DÍAS A PARTIR DE LA FECHA QUE ES RECIBIDA")
        c.drawString(50, 385, "RECARGO POR PAGO FRACCIONADO (MENSUAL: 6.5%, TRIMESTRAL: 5.5% Y")
        c.drawString(50, 373, "SEMESTRAL: 3.7%)")
        c.drawString(50, 343, "SEGUROS DEL VALLE, S. A., CON DOMICILIO EN AV. RÍO HONDO, NO. 1, COL. ALTAVISTA,")
        c.drawString(50, 331, "CP 08000, ALCALDÍA BENITO JUÁREZ, CDMX PONE A SU DISPOSICIÓN SU AVISO DE")
        c.drawString(50, 319, "PRIVACIDAD INTEGRAL EN LA PÁGINA WEB WWW.SEGUROSDELVALLE.COM.MX Y LE")
        c.drawString(50, 307, "INFORMA QUE SUS DATOS ESTÁN PROTEGIDOS Y SON UTILIZADOS SOLO PARA REGULAR")
        c.drawString(50, 295, "LOS DERECHOS Y OBLIGACIONES QUE SURGEN DE LA CELEBRACIÓN DE SU CONTRATO")
        c.drawString(50, 283, "DE SEGURO.")
        # Finalizar y preparar el buffer
        c.save()
        pdf_buffer.seek(0)
        # Subir PDF generado a S3
        s3.upload_fileobj(pdf_buffer, bucket_name, pdf_s3_key)
        print("✅ PDF subido exitosamente a:", f"s3://{bucket_name}/{pdf_s3_key}")

    complemento_hist_cotizaciones = pd.DataFrame({
        "Ticket": cotizacion["Ticket"],
        "Fecha de Inicio": cotizacion["Inicio"],
        "Mes": cotizacion["Mes"],
        "Oficina": cotizacion["Oficina"],
        "Contratante": cotizacion["Contratante"],
        "Agente": cotizacion["Agente"],
        "Prima": cotizacion["Prima"],
        "Evento": cotizacion["Evento"],
        "Tipo": cotizacion["Renovacion"],
        
    })

    complemento_hist_cotizaciones["Tipo"] = complemento_hist_cotizaciones["Tipo"].replace({
    "Si": "renovación",
    "No": "nuevo"
    })

    complemento_hist_cotizaciones["Prima"] = complemento_hist_cotizaciones["Prima"].replace({
    "La siniestralidad está desviada, consulte a un suscriptor": 0
    })

    #Agregamos el data frame en la parte inferior del archivo cotizaciones
    file_key = ruta_historico_cotizaciones
    response = s3.get_object(Bucket=nombre_bucket, Key=file_key)
    content = response['Body'].read()
    df_hist_cotizaciones = pd.read_excel(BytesIO(content), engine="openpyxl")
    df_completo = pd.concat([df_hist_cotizaciones, complemento_hist_cotizaciones], ignore_index=True)
    buffer = BytesIO()
    df_completo.to_excel(buffer, index=False, engine='openpyxl')
    buffer.seek(0)
    s3.upload_fileobj(buffer, nombre_bucket, file_key)


In [7]:
prima_experiencia_global(ruta_carpeta_base_datos, ruta_archivo_parametros, ruta_cuotas, ruta_carpeta_calculos, ruta_emisiones, ruta_historico_cotizaciones, nombre_bucket)

No se encontraron archivos en esa carpeta.


NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.